# LLC4320 Examples

In [ ]:
import xarray as xr
import intake
import numpy as np
%matplotlib inline

In [ ]:
coords = intake.cat.LLC4320_grid.to_dask().reset_coords()
coords

In [ ]:
sst = intake.cat.LLC4320_SST.to_dask()
sst

In [ ]:
u = intake.cat.LLC4320_SSU.to_dask()
v = intake.cat.LLC4320_SSV.to_dask()
ds = xr.merge([sst, u, v])
ds

In [ ]:
ds

In [ ]:
ds_natl = ds.sel(face=10)
coords_natl = coords.sel(face=10)
ds_natl

In [ ]:
import holoviews as hv
from holoviews.operation.datashader import regrid
hv.extension('bokeh')

In [ ]:
ocean_mask = coords_natl.hFacC.reset_coords(drop=True)>0
sst_natl = (ds_natl.SST.where(ocean_mask)
            .rename('SST'))
hv_image = hv.Dataset(sst_natl).to(hv.Image, kdims=['j', 'i'], dynamic=True)

%opts Image [width=900 height=600 invert_yaxis=True colorbar=True bgcolor='gray'] (cmap='RdBu_r')
regrid(hv_image, precompute=True)

In [ ]:
import xgcm
grid = xgcm.Grid(coords_natl.drop(['k', 'k_p1']), periodic=False)
grid

In [ ]:
eke = 0.5 * ( grid.interp(ds_natl.U, 'X', boundary='fill')**2 +
              grid.interp(ds_natl.V, 'Y', boundary='fill')**2
            ).where(ocean_mask).rename('EKE')
eke

In [ ]:
eke_image = hv.Dataset(np.log10(eke)).to(hv.Image, kdims=['j', 'i'], dynamic=True)

%opts Image [width=900 height=600 invert_yaxis=True colorbar=True bgcolor='gray' logz=False] (cmap='magma')
rg = regrid(eke_image, precompute=True).redim.range(EKE=(-4, 0))
rg

In [ ]:
ds_natl_daily = ds_natl.resample(time='D').mean()
ds_natl_daily

In [ ]:
from dask_kubernetes import KubeCluster
from dask.distributed import Client
cluster = KubeCluster(n_workers=5)
client = Client(cluster)
cluster

In [ ]:
eke_daily = 0.5 * (grid.interp(ds_natl_daily.U, 'X', boundary='fill')**2 +
                   grid.interp(ds_natl_daily.V, 'Y', boundary='fill')**2
                  ).where(ocean_mask).rename('EKE')
eke_daily

In [ ]:
eke_image = hv.Dataset(np.log10(eke_daily)).to(hv.Image, kdims=['j', 'i'], dynamic=True)

%opts Image [width=900 height=600 invert_yaxis=True colorbar=True bgcolor='gray' logz=False] (cmap='magma')
regrid(eke_image, precompute=True).redim.range(EKE=(-4, 0))

In [ ]:
zeta = (-grid.diff(ds_natl.U * coords_natl.dxC, 'Y', boundary='extend')
        +grid.diff(ds_natl.V * coords_natl.dyC, 'X', boundary='extend'))/coords_natl.rAz
zeta

In [ ]:
vort_image = hv.Dataset(zeta.rename('vort')).to(hv.Image, kdims=['j_g', 'i_g'], dynamic=True)

%opts Image [width=900 height=600 invert_yaxis=True colorbar=True bgcolor='gray' logz=False] (cmap='RdBu_r')
regrid(vort_image, precompute=True).redim.range(vort=(-1e-4, 1e-4))

### Try to reproduce Peter Cornillon's Gulf Stream Calculation

In [ ]:
xc = coords_natl.XC[:, 0].load()
yc = coords_natl.YC[0].load()
eke_daily.coords['yc'] = yc.reset_coords(drop=True)
eke_daily.coords['xc'] = xc.reset_coords(drop=True)
eke_daily_swap = eke_daily.swap_dims({'i': 'yc', 'j': 'xc'})
eke_daily_swap

In [ ]:
eke_gs = eke_daily_swap.sel(yc=slice(40, 30)).sel(xc=[-70, -68, -66], method='nearest')
eke_gs

In [ ]:
# takes a very long time on a small cluster
eke_gs.load()

In [ ]:
eke_gs.transpose().plot(col='xc', figsize=(12, 4))